In [1]:
%load_ext dotenv
%dotenv 

# What are we doing?

## Objectives 


* Build a data pipeline that downloads price data from the internet, stores it locally, transforms it into return data, and stores the feature set.
    - Getting the data.
    - Schemas and index in dask.

* Explore the parquet format.
    - Reading and writing parquet files.
    - Read datasets that are stored in distributed files.
    - Discuss dask vs pandas as a small example of big vs small data.
    
* Discuss the use of environment variables for settings.
* Discuss how to use Jupyter notebooks and source code concurrently. 
* Logging and using a standard logger.

## About the Data

+ We will download the prices for a list of stocks.
+ The source is Yahoo Finance and we will use the API provided by the library yfinance.


## Medallion Architecture

+ The architecture that we are thinking about is called Medallion by [DataBricks](https://www.databricks.com/glossary/medallion-architecture). It is an ELT type of thinking, although our data is well-structured.

![Medallion Architecture (DataBicks)](./images/02_medallion_architecture.png)

+ In our case, we would like to optimize the number of times that we download data from the internet. 
+ Ultimately, we will build a pipeline manager class that will help us control the process of obtaining and transforming our data.

![](./images/02_target_pipeline_manager.png)

# Download Data

Download the [Stock Market Dataset from Kaggle](https://www.kaggle.com/datasets/jacksoncrow/stock-market-dataset). Note that you may be required to register for a free account.

Extract all files into the directory: `./05_src/data/prices_csv/`

Your folder structure should include the following paths:

+ `05_src/data/prices_csv/etfs`
+ `05_src/data/prices_csv/stocks`


In [2]:
import pandas as pd
import os
import sys
from glob import glob

sys.path.append(os.getenv('SRC_DIR'))

from utils.logger import get_logger
_logs = get_logger(__name__)

A few things to notice in the code chunk above:

+ Libraries are ordered from high-level to low-level libraries from the package manager (pip in this case, but could be conda, poetry, etc.)
+ The command `sys.path.append("../05_src/)` will add the `../05_src/` directory to the path in the Notebook's kernel. This way, we can use our modules as part of the notebook.
+ Local modules are imported at the end. 
+ The function `get_logger()` is called with `__name__` as recommended by the documentation.

Now, to load the historical price data for stocks and ETFs, we could use:

In [3]:
import random

stock_files = glob(os.path.join(os.getenv('SRC_DIR'), "data/prices_csv/stocks/*.csv"))

random.seed(42)
stock_files = random.sample(stock_files, 60)

dt_list = []
for s_file in stock_files:
    _logs.info(f"Reading file: {s_file}")
    dt = pd.read_csv(s_file).assign(
        source = os.path.basename(s_file),
        ticker = os.path.basename(s_file).replace('.csv', ''),
        Date = lambda x: pd.to_datetime(x['Date'])
    )
    dt_list.append(dt)
stock_prices = pd.concat(dt_list, axis = 0, ignore_index = True)



2025-10-02 19:31:08,498, 4121356139.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/PAC.csv
2025-10-02 19:31:08,570, 4121356139.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/BOOM.csv
2025-10-02 19:31:08,570, 4121356139.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/BOOM.csv
2025-10-02 19:31:08,625, 4121356139.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/MNKD.csv
2025-10-02 19:31:08,625, 4121356139.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/MNKD.csv
2025-10-02 19:31:08,688, 4121356139.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/LGI.csv
2025-10-02 19:31:08,688, 4121356139.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/LGI.csv
2025-10-02 19:31:08,756, 4121356139.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/J.csv
2025-10-02 19:31:08,756, 4121356139.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/J.csv
2025-10-02 19:31:08,823, 412

Verify the structure of the `stock_prices` data:

In [4]:
stock_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256050 entries, 0 to 256049
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Date       256050 non-null  datetime64[ns]
 1   Open       256048 non-null  float64       
 2   High       256048 non-null  float64       
 3   Low        256048 non-null  float64       
 4   Close      256048 non-null  float64       
 5   Adj Close  256048 non-null  float64       
 6   Volume     256048 non-null  float64       
 7   source     256050 non-null  object        
 8   ticker     256050 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 17.6+ MB


We can subset our ticker data set using standard indexing techniques. A good reference for this type of data manipulation is Panda's [Documentation](https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-and-selecting-data) and [Cookbook](https://pandas.pydata.org/docs/user_guide/cookbook.html#cookbook-selection).

From the subset data frame, select one column and convert to list.

In [5]:
select_tickers = stock_prices['ticker'].unique().tolist()
select_tickers

['PAC',
 'BOOM',
 'MNKD',
 'LGI',
 'J',
 'MCX',
 'RNWK',
 'ZYNE',
 'TRVG',
 'LH',
 'IPAR',
 'NTIP',
 'UCBI',
 'NNVC',
 'FIT',
 'WABC',
 'MCI',
 'RLJ',
 'MTN',
 'PACQU',
 'GSM',
 'SND',
 'EAB',
 'GTHX',
 'SSYS',
 'EFT',
 'AYX',
 'EOG',
 'HBNC',
 'CTLT',
 'FIVN',
 'KN',
 'QTRX',
 'WEBK',
 'CQP',
 'GSH',
 'ANDE',
 'LMRKO',
 'TG',
 'QDEL',
 'ARL',
 'DZSI',
 'ETW',
 'UBP',
 'CHWY',
 'TX',
 'ENZ',
 'ABMD',
 'OFED',
 'RBNC',
 'MAG',
 'INSUU',
 'IDYA',
 'IBIO',
 'TRIB',
 'CXDC',
 'HIW',
 'CRAI',
 'WDR',
 'GHL']

# Storing Data in CSV



+ We have some data. How do we store it?
+ We can compare two options, CSV and Parqruet, by measuring their performance:

    - Time to save.
    - Space required.

In [6]:
def get_dir_size(path='.'):
    '''Returns the total size of files contained in path.'''
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total

In [7]:
import time
import shutil

In [8]:
temp = os.getenv("TEMP_DATA")
csv_dir = os.path.join(temp, "csv")
shutil.rmtree(csv_dir, ignore_errors=True)
stock_csv = os.path.join(csv_dir, "stock_px.csv")
os.makedirs(csv_dir, exist_ok=True)

In [9]:

start = time.time()
stock_prices.to_csv(stock_csv, index = False)
end = time.time()

_logs.info(f'Writing data ({stock_prices.shape}) to csv took {end - start} seconds.')
_logs.info(f'CSV file size { os.path.getsize(stock_csv)*1e-6 } MB')

2025-10-02 19:31:14,537, 473192941.py, 5, INFO, Writing data ((256050, 9)) to csv took 2.4330239295959473 seconds.
2025-10-02 19:31:14,538, 473192941.py, 6, INFO, CSV file size 28.123426 MB
2025-10-02 19:31:14,538, 473192941.py, 6, INFO, CSV file size 28.123426 MB


## Save Data to Parquet

### Dask 

We can work with with large data sets and parquet files. In fact, recent versions of pandas support pyarrow data types and future versions will require a pyarrow backend. The pyarrow library is an interface between Python and the Appache Arrow project. The [parquet data format](https://parquet.apache.org/) and [Arrow](https://arrow.apache.org/docs/python/parquet.html) are projects of the Apache Foundation.

However, Dask is much more than an interface to Arrow: Dask provides parallel and distributed computing on pandas-like dataframes. It is also relatively easy to use, bridging a gap between pandas and Spark. 

In [10]:
import dask.dataframe as dd

parquet_dir = os.path.join(temp, "parquet")
shutil.rmtree(parquet_dir, ignore_errors=True)
os.makedirs(parquet_dir, exist_ok=True)

In [11]:
# Check if required packages are installed and print their versions
try:
    import pandas as pd
    import dask
    import dask.dataframe as dd
    import pyarrow
    print('pandas:', pd.__version__)
    print('dask:', dask.__version__)
    print('pyarrow:', pyarrow.__version__)
except ImportError as e:
    print('ImportError:', e)

pandas: 2.3.2
dask: 2025.7.0
pyarrow: 19.0.0


In [12]:
px_dd = dd.from_pandas(stock_prices, npartitions = len(select_tickers))

start = time.time()
px_dd.to_parquet(parquet_dir, engine = "pyarrow")
end = time.time()

_logs.info(f'Writing dd ({stock_prices.shape}) to parquet took {end - start} seconds.')
_logs.info(f'Parquet file size { get_dir_size(parquet_dir)*1e-6 } MB')

2025-10-02 19:31:23,534, 817812245.py, 7, INFO, Writing dd ((256050, 9)) to parquet took 1.8489189147949219 seconds.
2025-10-02 19:31:23,535, 817812245.py, 8, INFO, Parquet file size 9.870593999999999 MB
2025-10-02 19:31:23,535, 817812245.py, 8, INFO, Parquet file size 9.870593999999999 MB


### Parquet files and Dask Dataframes

+ Parquet files are immutable: once written, they cannot be modified.
+ Dask DataFrames are a useful implementation to manipulate data stored in parquets.
+ Parquet and Dask are not the same: parquet is a file format that can be accessed by many applications and programming languages (Python, R, PowerBI, etc.), while Dask is a package in Python to work with large datasets using distributed computation.
+ **Dask is not for everything** (see [Dask DataFrames Best Practices](https://docs.dask.org/en/stable/dataframe-best-practices.html)). 

    - Consider cases suchas small to large joins, where the small dataframe fits in memory, but the large one does not. 
    - If possible, use pandas: reduce, then use pandas.
    - Pandas performance tips apply to Dask.
    - Use the index: it is beneficial to have a well-defined index in Dask DataFrames, as it may speed up searching (filtering) the data. A one-dimensional index is allowed.
    - Avoid (or minimize) full-data shuffling: indexing is an expensive operations. 
    - Some joins are more expensive than others. 

        * Not expensive:

            - Join a Dask DataFrame with a pandas DataFrame.
            - Join a Dask DataFrame with another Dask DataFrame of a single partition.
            - Join Dask DataFrames along their indexes.

        * Expensive:

            - Join Dask DataFrames along columns that are not their index.


# How do we store prices?

+ We can store our data as a single blob. This can be difficult to maintain, especially because parquet files are immutable.
+ Strategy: organize data files by ticker and date. Update only latest month.



In [13]:
# CLean up before start
PRICE_DATA = os.getenv("PRICE_DATA")
import shutil
if os.path.exists(PRICE_DATA):
    shutil.rmtree(PRICE_DATA)

In [14]:
stock_prices.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'ticker'],
      dtype='object')

In [15]:
for ticker in stock_prices['ticker'].unique():
    ticker_dt = stock_prices[stock_prices['ticker'] == ticker]
    ticker_dt = ticker_dt.assign(Year = ticker_dt.Date.dt.year)
    for yr in ticker_dt['Year'].unique():
        yr_dd = dd.from_pandas(ticker_dt[ticker_dt['Year'] == yr],2)
        yr_path = os.path.join(PRICE_DATA, ticker, f"{ticker}_{yr}")
        os.makedirs(os.path.dirname(yr_path), exist_ok=True)
        yr_dd.to_parquet(yr_path, engine = "pyarrow")
    

Why would we want to store data this way?

+ Easier to maintain. We do not update old data, only recent data.
+ We can also access all files as follows.

# Load, Transform and Save 

## Load

+ Parquet files can be read individually or as a collection.
+ `dd.read_parquet()` can take a list (collection) of files as input.
+ Use `glob` to get the collection of files.

In [16]:
from glob import glob

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

## Transform

+ This transformation step will create a *Features* data set. In our case, features will be stock returns (we obtained prices).
+ Dask dataframes work like pandas dataframes: in particular, we can perform groupby and apply operations.
+ Notice the use of [an anonymous (lambda) function](https://realpython.com/python-lambda/) in the apply statement.

In [17]:
dd_shift = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

/tmp/ipykernel_3093/102405636.py:1: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result

  dd_shift = dd_px.groupby('ticker', group_keys=False).apply(


In [18]:
dd_rets = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

## Lazy Exection

What does `dd_rets` contain?

In [19]:
dd_rets

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Returns
npartitions=60,,,,,,,,,,,
ABMD,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,float64,float64
ANDE,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
ZYNE,...,...,...,...,...,...,...,...,...,...,...
ZYNE,...,...,...,...,...,...,...,...,...,...,...


+ Dask is a lazy execution framework: commands will not execute until they are required. 
+ To trigger an execution in dask use `.compute()`.

In [20]:
dd_rets.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Returns
ticker,,,,,,,,,,,
ABMD,2005-01-03,15.87,15.970,14.970,15.01,15.01,213500.0,ABMD.csv,2005,NaN,NaN
ABMD,2005-01-04,15.18,15.200,14.760,14.98,14.98,140300.0,ABMD.csv,2005,15.01,-0.001999
ABMD,2005-01-05,14.75,15.100,14.210,14.69,14.69,160600.0,ABMD.csv,2005,14.98,-0.019359
ABMD,2005-01-06,14.49,14.790,14.270,14.52,14.52,194900.0,ABMD.csv,2005,14.69,-0.011572
ABMD,2005-01-07,14.56,14.560,13.740,14.29,14.29,97800.0,ABMD.csv,2005,14.52,-0.015840
...,...,...,...,...,...,...,...,...,...,...,...
ZYNE,2020-03-26,3.86,4.040,3.800,3.94,3.94,400700.0,ZYNE.csv,2020,3.83,0.028721
ZYNE,2020-03-27,3.84,4.000,3.630,3.81,3.81,383100.0,ZYNE.csv,2020,3.94,-0.032995
ZYNE,2020-03-30,3.83,3.892,3.510,3.67,3.67,321100.0,ZYNE.csv,2020,3.81,-0.036745


## Save

+ Apply transformations to calculate daily returns
+ Store the enriched data, the silver dataset, in a new directory.
+ Should we keep the same namespace? All columns?

In [21]:
# CLean up before save
FEATURES_DATA = os.getenv("FEATURES_DATA")
if os.path.exists(FEATURES_DATA):
    shutil.rmtree(FEATURES_DATA)
dd_rets.to_parquet(FEATURES_DATA, overwrite = True)

# Optional: from Jupyter to Command Line

+ We have drafted our code in a Jupyter Notebook. 
+ Finalized code should be written in Python modules.

## Object Oriented vs Functional Programming

+ We can use classes to keep parameters and functions together.
+ We *could* use Object Oriented Programming, but parallelization of data manipulation and modelling tasks benefit from *Functional Programming*.
+ An Idea: 

    - [Data Oriented Programming](https://blog.klipse.tech/dop/2022/06/22/principles-of-dop.html).
    - Use the class to bundle together parameters and functions.
    - Use stateless operations and treat all data objects as immutable (we do not modify them, we overwrite them).
    - Take advantage of [`@staticmethod`](https://realpython.com/instance-class-and-static-methods-demystified/).

The code is in `./05_src/stock_prices/data_manager.py`.

Our original design was:

![](./images/02_target_pipeline_manager.png)



In [22]:
from stock_prices.data_manager import DataManager
dm = DataManager()

Download all prices.

In [23]:
dm.process_sample_files()

2025-10-02 19:32:04,746, data_manager.py, 53, INFO, Processing sample of tickers
2025-10-02 19:32:04,749, data_manager.py, 64, INFO, Getting file list from ../../05_src/data/prices_csv/
2025-10-02 19:32:04,749, data_manager.py, 64, INFO, Getting file list from ../../05_src/data/prices_csv/
2025-10-02 19:32:04,776, data_manager.py, 66, INFO, Found 8050 files in ../../05_src/data/prices/
2025-10-02 19:32:04,777, data_manager.py, 74, INFO, Selecting sample of files
2025-10-02 19:32:04,778, data_manager.py, 80, INFO, Selected 30 files
2025-10-02 19:32:04,778, data_manager.py, 90, INFO, Processing file ../../05_src/data/prices_csv/stocks/SHIP.csv
2025-10-02 19:32:04,779, data_manager.py, 118, INFO, Reading file: ../../05_src/data/prices_csv/stocks/SHIP.csv
2025-10-02 19:32:04,776, data_manager.py, 66, INFO, Found 8050 files in ../../05_src/data/prices/
2025-10-02 19:32:04,777, data_manager.py, 74, INFO, Selecting sample of files
2025-10-02 19:32:04,778, data_manager.py, 80, INFO, Selected 3

Finally, add features to the data set and save to a *feature store*.

In [24]:
dm.featurize()

2025-10-02 19:32:12,314, data_manager.py, 131, INFO, Creating features data.
2025-10-02 19:32:12,315, data_manager.py, 141, INFO, Loading price data from ../../05_src/data/prices/
2025-10-02 19:32:12,315, data_manager.py, 141, INFO, Loading price data from ../../05_src/data/prices/
2025-10-02 19:32:12,515, data_manager.py, 150, INFO, Creating features
/workspaces/production/01_materials/labs/../../05_src/stock_prices/data_manager.py:154: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result

  .apply(
2025-10-02 19:32:12,523, data_manager.py, 175, INFO, Saving features to ../../05_src/data/features/stock_features
2025-10-02 19:32:12,515, data_manager.py, 150, INFO, Creating features
/workspaces/production/01_materials/labs/../../05_src/stock_prices/data_manager